## Deep Learning for Mortality Prediction (DLMP)

In [1]:
#%run data_preparation/split_data.py

### Import packages 

In [36]:
import tensorflow as tf
import numpy as np
import os as os
import matplotlib.pyplot as plt
import seaborn as sns
import keras
tfkl = tf.keras.layers
import math

### Import functions

In [37]:
import training_functions
import importlib

importlib.reload(training_functions)

<module 'training_functions' from '/home/ppark/repos/deep-mort/code/training_functions.py'>

### Import data

#### State data

In [38]:
state_training = np.loadtxt('../data/state_training.txt')
state_test = np.loadtxt('../data/state_test.txt')

#### Country data

In [39]:
country_training = np.loadtxt('../data/country_training_new.txt')
country_test = np.loadtxt('../data/country_test_new.txt')

#### Combined data

In [40]:
combined_training = np.loadtxt('../data/combined_training_new.txt')
combined_test = np.loadtxt('../data/combined_test_new.txt')

In [41]:
geos_key = np.load('../data/geos_key_new.npy')

In [42]:
geo_dict = {int(code): geo for geo, code in geos_key}

#### Single population models

Below, I am training indiviudal models for each population. I am only saving mses rather than predictions since the prediction data would be too large to store. 

In [43]:
n_pop = np.unique(country_test[:, 0])
geo_dim = int(max(n_pop) + 1)
geo_dim

90

In [1]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPU devices:", tf.config.list_physical_devices('GPU'))

# Optional: Set memory growth to prevent over-allocation
for gpu in tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)


2025-07-17 10:42:39.283342: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow version: 2.13.1
Built with CUDA: True
GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [34]:
for s in range(1,3):
    for j in range(0,2):
        for i in range(50, 51):
            country_index = i
            gender_index = j
            single_country_training = country_training[(country_training[:,0] == country_index) & (country_training[:,1] == gender_index)]
            single_country_test = country_test[(country_test[:,0] == country_index) & (country_test[:,1] == gender_index)]

            single_country_training_prepped = training_functions.prep_data(single_country_training, mode="train", changeratetolog=True)
            single_country_test_prepped = training_functions.prep_data(single_country_test, mode="test", changeratetolog=True)
            single_geo_dim = country_index + 1

            print(f"Running model {geo_dict[i]}: Gender {j}")

            model_single, loss_single = training_functions.run_deep_model(single_country_training_prepped, 
                                                                          single_country_test_prepped, 
                                                                          single_geo_dim, epochs=30, 
                                                                          steps_per_epoch=500, lograte=True)
        
            tc = single_country_test  
            years = (tc[:,2] - 1959) / 60
            ages  = tc[:,3]
            geos  = tc[:,0].astype(np.int32)
            genders = tc[:,1]

            test_input_features = (
                tf.convert_to_tensor(years, dtype=tf.float32),
                tf.convert_to_tensor(ages, dtype=tf.float32),
                tf.convert_to_tensor(geos, dtype=tf.int32),      # ensure ints
                tf.convert_to_tensor(genders, dtype=tf.float32),
            )
            
            test_predictions = model_single.predict(test_input_features)


            inputs_test = np.delete(single_country_test, 4, axis=1)
            test_predictions = np.column_stack((inputs_test, test_predictions))

            geo_name = geo_dict[i]

            model_single.save(f"../models/{geo_name}_gender_{j}.keras")   

            np.savez_compressed(f"../data/single_preds/{geo_name}_gender_{j}_iter_{s}.npz", test_predictions) 

print(f"Finished training for {s} iterations.")

Running model AUS: Gender 0
hello
Epoch 1/30
500/500 - 12s - loss: 5.5942 - val_loss: 0.4289 - lr: 0.0010 - 12s/epoch - 24ms/step
Epoch 2/30
500/500 - 4s - loss: 0.3910 - val_loss: 0.1492 - lr: 0.0010 - 4s/epoch - 9ms/step
Epoch 3/30
500/500 - 4s - loss: 0.2936 - val_loss: 0.0980 - lr: 0.0010 - 4s/epoch - 8ms/step
Epoch 4/30
500/500 - 4s - loss: 0.2268 - val_loss: 0.1172 - lr: 0.0010 - 4s/epoch - 8ms/step
Epoch 5/30
500/500 - 4s - loss: 0.1817 - val_loss: 0.0450 - lr: 0.0010 - 4s/epoch - 8ms/step
Epoch 6/30
500/500 - 4s - loss: 0.1519 - val_loss: 0.1110 - lr: 0.0010 - 4s/epoch - 7ms/step
Epoch 7/30
500/500 - 4s - loss: 0.1301 - val_loss: 0.0584 - lr: 0.0010 - 4s/epoch - 7ms/step
Epoch 8/30
500/500 - 4s - loss: 0.1127 - val_loss: 0.0366 - lr: 0.0010 - 4s/epoch - 7ms/step
Epoch 9/30
500/500 - 4s - loss: 0.1015 - val_loss: 0.0421 - lr: 0.0010 - 4s/epoch - 9ms/step
Epoch 10/30
500/500 - 4s - loss: 0.0916 - val_loss: 0.0428 - lr: 0.0010 - 4s/epoch - 8ms/step
Epoch 11/30
500/500 - 4s - loss:

#### All Country Model

In [34]:
# prep data
country_train_prepped = training_functions.prep_data(country_training, mode="train", changeratetolog=True)
country_test_prepped = training_functions.prep_data(country_test, mode="test", changeratetolog=True)

In [35]:
# get the proper geography input dimension for model set up 
unique_vals = tf.unique(country_training[:, 0]).y
country_geo_dim = np.array(tf.size(unique_vals)).item()
country_geo_dim = country_geo_dim + 50
country_geo_dim

90

In [37]:
math.ceil(len(country_training)/ 256)

1405

In [ ]:
# run country model
for i in range(1,6):

    model_country, loss_info_country = training_functions.run_deep_model(country_train_prepped, country_test_prepped, country_geo_dim, 
                                                                         epochs=20, steps_per_epoch=1405, lograte=True)

    training_input_features = (tf.convert_to_tensor((country_training[:,2] - 1959) / 60, dtype=tf.float32),  # Normalized year
                            tf.convert_to_tensor(country_training[:,3], dtype=tf.float32),  # Age
                            tf.convert_to_tensor(country_training[:,0], dtype=tf.float32),  # Geography
                            tf.convert_to_tensor(country_training[:,1], dtype=tf.float32))  # Gender
    
    test_input_features = (tf.convert_to_tensor((country_test[:,2] - 1959) / 60, dtype=tf.float32),  # Normalized year
                        tf.convert_to_tensor(country_test[:,3], dtype=tf.float32),  # Age
                        tf.convert_to_tensor(country_test[:,0], dtype=tf.float32),  # Geography
                        tf.convert_to_tensor(country_test[:,1], dtype=tf.float32))  # Gender

    training_predictions = model_country.predict(training_input_features)

    test_predictions = model_country.predict(test_input_features)

    inputs = np.delete(country_training, 4, axis=1)
    training_predictions = np.column_stack((inputs, training_predictions))
    inputs_test = np.delete(country_test, 4, axis=1)
    test_predictions = np.column_stack((inputs_test, test_predictions))

    model_country.save(f"../models/dl_country_model_{i}.keras")

    np.savetxt(f"../data/dl_country_forecast{i}.txt", training_predictions)
    np.savetxt(f"../data/dl_country_forecast{i}.txt", test_predictions)    

    print(f"Iteration {i} complete")

Epoch 1/20
1405/1405 - 11s - 8ms/step - loss: 2.0501 - val_loss: 0.2499 - learning_rate: 0.0010
Epoch 2/20
1405/1405 - 9s - 7ms/step - loss: 0.3321 - val_loss: 0.2121 - learning_rate: 0.0010
Epoch 3/20
1405/1405 - 10s - 7ms/step - loss: 0.2541 - val_loss: 0.1817 - learning_rate: 0.0010
Epoch 4/20
1405/1405 - 13s - 9ms/step - loss: 0.2135 - val_loss: 0.1808 - learning_rate: 0.0010
Epoch 5/20
1405/1405 - 9s - 7ms/step - loss: 0.2010 - val_loss: 0.1987 - learning_rate: 0.0010
Epoch 6/20
1405/1405 - 9s - 7ms/step - loss: 0.1932 - val_loss: 0.2210 - learning_rate: 0.0010
Epoch 7/20
1405/1405 - 11s - 8ms/step - loss: 0.1786 - val_loss: 0.2357 - learning_rate: 0.0010
Epoch 8/20
1405/1405 - 10s - 7ms/step - loss: 0.1682 - val_loss: 0.1942 - learning_rate: 2.5000e-04
Epoch 9/20
1405/1405 - 9s - 7ms/step - loss: 0.1720 - val_loss: 0.1717 - learning_rate: 2.5000e-04
Epoch 10/20
1405/1405 - 10s - 7ms/step - loss: 0.1637 - val_loss: 0.1915 - learning_rate: 2.5000e-04
Epoch 11/20
1405/1405 - 10s - 7

### Combined (all country / all state) DL Model

In [53]:
# convert combined data to tensors and other prep
combined_train_prepped = training_functions.prep_data(combined_training, mode="train", changeratetolog=True)
combined_test_prepped = training_functions.prep_data(combined_test, mode="test", changeratetolog=True)

In [54]:
# get the proper geography input dimension for model set up 
unique_vals = tf.unique(combined_training[:, 0]).y
combined_geo_dim = np.array(tf.size(unique_vals)).item()
print(combined_geo_dim)

90


In [56]:
# run combined model
for i in range(1,6):

    model_combined, loss_info_combined = training_functions.run_deep_model(combined_train_prepped, combined_test_prepped, 
                                                                           combined_geo_dim, epochs=20, steps_per_epoch=2000,
                                                                           lograte=True)

    training_input_features = (tf.convert_to_tensor((combined_training[:,2] - 1959) / 60, dtype=tf.float32),  # Normalized year
                            tf.convert_to_tensor(combined_training[:,3], dtype=tf.float32),  # Age
                            tf.convert_to_tensor(combined_training[:,0], dtype=tf.float32),  # Geography
                            tf.convert_to_tensor(combined_training[:,1], dtype=tf.float32))  # Gender
    
    test_input_features = (tf.convert_to_tensor((combined_test[:,2] - 1959) / 60, dtype=tf.float32),  # Normalized year
                        tf.convert_to_tensor(combined_test[:,3], dtype=tf.float32),  # Age
                        tf.convert_to_tensor(combined_test[:,0], dtype=tf.float32),  # Geography
                        tf.convert_to_tensor(combined_test[:,1], dtype=tf.float32))  # Gender

    training_predictions = model_combined.predict(training_input_features)

    test_predictions = model_combined.predict(test_input_features)

    inputs = np.delete(combined_training, 4, axis=1)
    training_predictions = np.column_stack((inputs, training_predictions))
    inputs_test = np.delete(combined_test, 4, axis=1)
    test_predictions = np.column_stack((inputs_test, test_predictions))

    model_combined.save(f"../models/combined_model_{i}.keras")

    np.savetxt(f"../data/combined_training_predictions{i}.txt", training_predictions)
    np.savetxt(f"../data/combined_test_predictions{i}.txt", test_predictions)    

    print(f"Iteration {i} complete")

Epoch 1/20
2000/2000 - 19s - 9ms/step - loss: 1.5262 - val_loss: 0.2362 - learning_rate: 0.0010
Epoch 2/20
2000/2000 - 16s - 8ms/step - loss: 0.2855 - val_loss: 0.2518 - learning_rate: 0.0010
Epoch 3/20
2000/2000 - 16s - 8ms/step - loss: 0.2300 - val_loss: 0.2428 - learning_rate: 0.0010
Epoch 4/20
2000/2000 - 14s - 7ms/step - loss: 0.2115 - val_loss: 0.2219 - learning_rate: 0.0010
Epoch 5/20
2000/2000 - 14s - 7ms/step - loss: 0.1956 - val_loss: 0.2163 - learning_rate: 0.0010
Epoch 6/20
2000/2000 - 14s - 7ms/step - loss: 0.1900 - val_loss: 0.1822 - learning_rate: 0.0010
Epoch 7/20
2000/2000 - 13s - 7ms/step - loss: 0.1857 - val_loss: 0.1922 - learning_rate: 0.0010
Epoch 8/20
2000/2000 - 22s - 11ms/step - loss: 0.1839 - val_loss: 0.1892 - learning_rate: 0.0010
Epoch 9/20
2000/2000 - 18s - 9ms/step - loss: 0.1836 - val_loss: 0.2220 - learning_rate: 0.0010
Epoch 10/20
2000/2000 - 14s - 7ms/step - loss: 0.1757 - val_loss: 0.1742 - learning_rate: 2.5000e-04
Epoch 11/20
2000/2000 - 14s - 7ms/